In [59]:
import datalab.storage as storage
from io import BytesIO
import io
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from skimage import filters
from skimage.filters import try_all_threshold
from skimage.filters import threshold_otsu, rank, threshold_yen
from skimage.restoration import denoise_wavelet
from skimage.filters import scharr
from skimage.morphology import binary_closing
from skimage.morphology import disk
from skimage.filters.rank import median
from PIL import Image
from skimage.morphology import binary_erosion
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.morphology import erosion, dilation
from skimage.morphology import disk
from skimage import data, color, img_as_ubyte
from skimage.morphology import opening, closing
from skimage.filters import roberts
from skimage import exposure
from skimage.restoration import denoise_wavelet
from skimage.morphology import binary_closing
from skimage.color import rgb2gray
from skimage.color import convert_colorspace
from PIL import Image, ImageDraw, ImageColor
from scipy.misc import toimage
import pandas as pd

In [60]:
def bloodSegmentationModified(img_orig):
  img_orig = rgb2gray(img_orig)
  blur1 = gaussian_filter(img_orig, 24)
  min_img = np.minimum(img_orig, blur1)
  blur2 = gaussian_filter(min_img, 1)
  edges = scharr(blur2)
  threshold_global_otsu = threshold_otsu(edges)
  global_otsu = edges >= threshold_global_otsu
  closed = binary_closing(global_otsu)
  eroded = binary_erosion(closed)
  updated = median(eroded, disk(3))
  no_noise = denoise_wavelet(updated, multichannel=False)
  return no_noise

In [61]:
def bloodSegmentation(img_orig):
  black_white = rgb2gray(img_orig)
  blur1 = gaussian_filter(black_white, 24)
  min_img = np.minimum(black_white, blur1)
  blur2 = gaussian_filter(min_img, 1)
  edges = scharr(blur2)
  threshold_global_otsu = threshold_otsu(edges)
  global_otsu = edges >= threshold_global_otsu
  noisy = global_otsu
  no_noise = denoise_wavelet(noisy, multichannel=False)
  return no_noise

In [62]:
def opticDiskSegmentation(img_orig):
  selem = disk(5)
  hsv_img = convert_colorspace(img_orig, 'RGB', 'HSV')
  sat = hsv_img[:, : ,2]
  eroded = dilation(sat, selem)
  selem = disk(16)
  opened = opening(eroded, selem)
  closed = closing(opened, selem)
  edges = roberts(closed)
  hist = exposure.equalize_hist(edges)
  noisy = hist
  no_noise = denoise_wavelet(noisy, multichannel=False)
  thresh = threshold_yen(no_noise)
  global_yen = no_noise >= thresh
  selem = disk(30)
  OD_final = binary_closing(global_yen, selem)
  img = Image.frombytes('L', (OD_final.shape[1], OD_final.shape[0]), OD_final.tobytes())
  width, height = img.size
  center = (int(0.5 * width), int(0.5 * height))
  ImageDraw.floodfill(img, center, 255)
  to_label = np.array(img)
  inverted = np.array(to_label)
  to_label = 255 - inverted
  label_image = label(to_label)
  props = regionprops(label_image)
  props = sorted(props, key=lambda region: region.area, reverse=True)
  if len(props) < 2:
    return None, None, None, None
  label_num = props[1].label
  disc = label_image == label_num
  OD_center = props[1].centroid
  OD_area = props[1].filled_area
  OD_diam = props[1].equivalent_diameter
  return OD_center, OD_area, OD_diam, disc

In [63]:
def getFeatures(optic_disk, blood_seg):
  intersect = np.logical_and(optic_disk > 0, blood_seg > 0)
  n_white_pix_overlap = np.sum(intersect > 0)
  n_white_pix_OD = np.sum(optic_disk >0)
  n_white_pix_blood = np.sum(blood_seg > 0)
  return n_white_pix_overlap, n_white_pix_blood

In [66]:
for i in range(1, 4):
  for j in range(1, 5):
    pre = "Base" +str(i)+str(j)
    print(pre)
    if pre == "Base11" or  pre=="Base12":
      continue
    img_name = []
    overlap = []
    area_blood = []
    OD_centers = [] 
    OD_areas = []
    OD_diams = []
    bucket = storage.Bucket('bmi260-dr')
    count = 0
    for image in bucket.items(prefix=pre):
      count += 1
      if count % 10 == 0:
        print(count)
      uri = image.uri
      file_name = uri.split("gs://bmi260-dr/{0}/".format(pre))[1]
      if "xls" in str(file_name) or "Annotation" in str(file_name):
        continue
      %gcs read --object $uri --variable data
      img_orig = Image.open(io.BytesIO(data))
      try:
          img_orig = Image.open(io.BytesIO(data))
      except IOError:
          print 'error opening file {} '.format(file_name)
          continue
      img_orig = np.array(img_orig)
      segmented = bloodSegmentation(img_orig)
      OD_center, OD_area, OD_diam, disc = opticDiskSegmentation(img_orig)
      if OD_center == None:
        continue
      n_white_pix_overlap, n_white_pix_blood = getFeatures(disc, segmented)
      img_name.append(file_name)
      overlap.append(n_white_pix_overlap)
      OD_centers.append(OD_center)
      OD_areas.append(OD_area)
      OD_diams.append(OD_diam)
      area_blood.append(n_white_pix_blood)
    data_for_frame = {'IMG':img_name, 'BLOOD_IN_DISK':overlap, 'OPTIC_DISK_AREA':OD_areas, 'BLOOD_AREA':area_blood, 'OD_center':OD_centers, 'OD_diams':OD_diams}
    data_df = pd.DataFrame(data=data_for_frame)
    label_data = bucket.item('{0}/Annotation_{1}.csv'.format(pre, pre))
    uri = label_data.uri
    %gcs read --object $uri --variable data2
    label_df = pd.read_csv(BytesIO(data2))
    label_df = label_df[['Image name', 'Retinopathy grade']]
    merged = label_df.merge(data_df, left_on='Image name', right_on='IMG', how='inner')
    dataset = [i for k in range(merged.shape[0])]
    merged['dataset'] = dataset
    bucket.item('blood_info_{}_with_label.csv'.format(pre)).write_to(merged.to_csv(),'text/csv')

Base11
Base12
Base13
10
20
30
40
50
60
70
80
90
100
Base14
10
20
30
40
50
60
70
80
90
100
Base21
10
20
30
40
50
60
70
80
90
100
Base22
10
20
30
40
50
60
70
80
90
100
Base23
10
20
30
40
50
60
70
80
90
100
Base24
10
20
30
40
50
60
70
80
90
100
Base31
10
20
30
40
50
60
70
80
90
100
Base32
10
20
30
40
50
60
70
80
90
100
Base33
10
20
30
40
50
60
70
80
90
100
Base34
10
20
30
40
50
60
70
80
90
100
